In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

# Range of days you want to see
StartDate = datetime(year,month,day,14,0) # Never mind with numbers 14 and 0. Just put year, month, and day in.
EndDate = datetime(2010,1,3,14,0)

# Range of hours that you want to see everyday
StartHour = 8
EndHour = 18

DateRange = (EndDate - StartDate).days
HourRange = EndHour - StartHour

# Access the website
Date = str(StartDate).replace("-",".").replace(" ",".")[:-6]
url = "https://www.weather.go.kr/weather/observation/currentweather.jsp?auto_man=m&stn=0&type=t11&reg=100&tm="+Date+"%3A00&x=16&y=5"
wd = webdriver.Chrome(ChromeDriverManager().install())
wd.get(url)

# Scrape the html file 
req = wd.page_source
soup = BeautifulSoup(req, 'html.parser')

# Make a Resualtant diagram to fill in 
table = soup.select('table') 
table = str(table)
table_df_list = pd.read_html(table)
table_df = table_df_list[0]

Result = table_df.iloc[:,:2]
Result.rename(columns = {'01H' : 'SubZeroRate'}, inplace = True)
Result["SubZeroRate"] = 0

# Fill in the diagram
time = StartDate
while((time - EndDate).days < 1):
    req = wd.page_source
    soup = BeautifulSoup(req,"html.parser")
    
    # Find the table at the website
    table = soup.select('table.table_develop3') 
    table = str(table)
    table_df_list = pd.read_html(table)
    table_df = table_df_list[0]

    # Show the current date that the program is digging in 
    date = soup.select_one('#observation_text')
    date = date["value"]
    print(date[:-6])
    
    # Extract the specific range of hours that you want to look at
    Worktime = pd.concat([table_df.iloc[:,0],table_df.iloc[:,StartHour:EndHour+1]],axis = 1)

    # Filter out the hours of which temperature is over zero.
    Cold = Worktime[Worktime.iloc[:,1:]<0]
    Cold = np.array(Cold)

    # Calculate the overall subzero hours within each region
    for i in range(len(Cold)):
        num = (Cold[i]<0).sum()
        Result.iloc[i,1] += num
        
    time += timedelta(days = 1)
    Tommorrow = wd.find_element_by_xpath('//*[@id="content_weather"]/div[2]/form/fieldset[2]/ul/li[5]/a')
    Tommorrow.click()
        
for i in range(len(Result)):
    Result.iloc[i,1] = Result.iloc[i,1]/36500

print(Result)
#Result.to_excel('WeatherKorea.xlsx')